<a href="https://colab.research.google.com/github/hopesofbuzzy/URFU_adii/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B8/11/06_PromptEng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Работа с LLM GigaChat

[GigaChat](https://developers.sber.ru/docs/ru/gigachain/tools/python/gigachat) — это Python-библиотека для работы с `REST API GigaChat`. Она является частью [`GigaChain`](https://github.com/Scicommunity/gigachain) и входит в состав `langchain-gigachat` — партнерского пакета opensource-фреймворка [`LangChain`](https://www.langchain.com/).

[Библиотека](https://gitverse.ru/ai-forever/gigachain) управляет авторизацией запросов и предоставляет все необходимые методы для работы с `API`. Кроме этого она поддерживает:

* обработку потоковой передачи токенов;
* работу с функциями;
* создание эмбеддингов;
* работу в синхронном или в асинхронном режиме.

СМ. Также
* [туторилаы по `GigaChat`](https://giga.chat/help/tutorials)
* [примеры работы с библиотекой `GigaChat`](https://github.com/ai-forever/gigachat/tree/main/examples)
* [некоторые неофициальные туториалы по `GigaChat`](https://github.com/trashchenkov/gigachat_tutorials)
* [репозиторий `GigaChain`](https://gitverse.ru/ai-forever/gigachain)
* [`langchain` tutorials](https://github.com/gkamradt/langchain-tutorials)
* Статьи на Хабр [тут](https://habr.com/ru/articles/906584/), [тут](https://habr.com/ru/companies/sberbank/articles/941340/) и [тут](https://habr.com/ru/companies/sberdevices/articles/794773/)

Для получения доступа к [GigaChat](https://developers.sber.ru/docs/ru/gigachain/tools/python/gigachat) нужно пройти [авторизацию тут](https://developers.sber.ru/studio/login) или другим [способом авторизации](https://developers.sber.ru/docs/ru/gigachain/tools/python/gigachat#sposoby-avtorizatsii).

In [10]:
!pip install gigachat
!pip install ddgs

In [2]:
from gigachat import GigaChat
from gigachat.models import Chat, Function, FunctionParameters, Messages, MessagesRole


In [3]:
import json
from ddgs import DDGS

from google.colab import userdata

Прочитаем токен

In [ ]:
token = open('authGigaChat.txt').read().strip()

In [4]:
client_id = userdata.get('SBER_ID')
secret = userdata.get('SBER_SECRET')
auth = userdata.get('SBER_AUTH')

import base64
credentials = f"{client_id}:{secret}"
print(credentials)
encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')


encoded_credentials == auth

019a6bdb-607f-745f-aafe-84a8fed0da0b:a2c04575-4ef1-480e-954e-afd1d3ac34ce


True

Проверим что все работает для этого попробуем задать простой вопрос __пользователя__ к
__модели__

In [ ]:
MESSAGE = "Дай определение и Расскажи в 3 коротких пунктах об особенностях prompt engineering, "

TypeError: GigaChatSyncClient.get_models() missing 1 required positional argument: 'self'

In [ ]:
with GigaChat(credentials=authverify_ssl_certs=False) as giga:
    for model in giga.get_models().data:
      print(model.id_)

GigaChat
GigaChat-2
GigaChat-2-Max
GigaChat-2-Pro
GigaChat-Max
GigaChat-Max-preview
GigaChat-Plus
GigaChat-Pro
GigaChat-Pro-preview
GigaChat-preview
Embeddings
Embeddings-2
EmbeddingsGigaR
GigaEmbeddings-3B-2025-09


In [ ]:
with GigaChat(credentials=auth, model="GigaChat-2", verify_ssl_certs=False) as giga:
    response = giga.chat(MESSAGE)

print(response.choices[0].message.content)

**Prompt Engineering (инженерия запросов)** — методология составления чётких, лаконичных и понятных инструкций для нейросетей, направленная на получение точного и качественного результата от модели ИИ.

### Особенности Prompt Engineering:

1. **Сжатость формулировок:**   
   Короткость и конкретика запроса позволяют нейросети лучше понимать цель и быстрее выдавать нужный результат.
   
2. **Использование структуры и контекста:**  
   Четкая структура подсказки и включение контекста помогают избежать двусмысленности и уточняют ожидания модели относительно типа желаемого ответа.

3. **Гибкость в интерпретации:**  
   Инженеры запросов используют различные техники уточнения (например, добавление тегов, директив и пояснений), чтобы повлиять на поведение нейросети и получить наиболее подходящий отклик.


----------------------------------------------------

В Использованной модели

|Параметр|Обязательный|Описание|
|---|---|---|
|`credentials`|да|Ключ авторизации для обмена сообщениями с `GigaChat API`. Ваш токен доступа из личного кабинета Сбера.|
| | |Ключ авторизации содержит информацию о версии `API,` к которой выполняются запросы. Если вы используете версию `API` для ИП или юрлиц, укажите это явно в параметре `scope`|
|`verify_ssl_certs`|нет|Отключение проверки `ssl`-сертификатов.|
| | | |
| | |[Для обращения к `GigaChat API` нужно установить корневой сертификат НУЦ Минцифры.](https://developers.sber.ru/docs/ru/gigachat/certificates)|
| | | |
| | |Используйте параметр ответственно, так как отключение проверки сертификатов снижает безопасность обмена данными|
|`scope`|нет|Версия `API`, к которой будет выполнен запрос. По умолчанию запросы передаются в версию для физических лиц. Возможные значения:|
| | |`GIGACHAT_API_PERS` — версия `API` для физических лиц;|
| | |`GIGACHAT_API_B2B` — версия `API` для ИП и юрлиц при работе по предоплате.|
| | |`GIGACHAT_API_CORP` — версия `API` для ИП и юрлиц при работе по постоплате.|
|`model`|нет|необязательный параметр, в котором можно явно задать модель `GigaChat`. Вы можете посмотреть список доступных моделей с помощью метода `get_models()`, который выполняет запрос `GET /models`.|
| | | |
| | |[Стоимость запросов к разным моделям отличается. Подробную информацию о тарификации запросов к той или иной модели вы ищите в официальной документации](https://developers.sber.ru/docs/ru/gigachat/api/tariffs)|
|base_url|нет|[Адрес API. По умолчанию запросы отправляются по адресу https://gigachat.devices.sberbank.ru/api/v1/, но если вы хотите использовать модели в раннем доступе, укажите адрес https://gigachat-preview.devices.sberbank.ru/api/v1](https://developers.sber.ru/docs/ru/gigachat/models/preview-models)|

Также синтаксис выход модели - `markdown`, поэтому сделаем его более читаемым

In [ ]:
from IPython.display import display, Markdown

In [ ]:
with GigaChat(credentials=token, verify_ssl_certs=False) as giga:
    response = giga.chat(MESSAGE)
    content = response.choices[0].message.content
    display(Markdown("<blockquote>\n\n"+content))


<blockquote>

**Prompt Engineering (инжиниринг подсказок)** — это процесс тщательной разработки и настройки входной информации (подсказки), подаваемой модели машинного обучения (например, языковой модели), чтобы получить оптимальный результат выполнения поставленных задач.

### Особенности Prompt Engineering:

1. **Учет контекста**:  
   Формулировка подсказки должна учитывать специфику поставленной задачи, включая тонкости формулировок, используемые термины и требуемый контекст.

2. **Структурирование запроса**:  
   Грамотная структура подсказки помогает модели эффективно понимать и обрабатывать запросы, снижая вероятность ошибок и улучшая качество результата.

3. **Контроль выдачи**:  
   Правильный выбор инструкций и шаблонов позволяет добиться нужной длины ответа, его точности и стилистики.

______________________________________________________

Перейдем к выбору моделей. Актуальный список моделей можно найти [тут](https://developers.sber.ru/docs/ru/gigachat/models). Модели могут отличатся качеством и разнообразием ответов.

In [ ]:
model = GigaChat(
    model="GigaChat-2",
    credentials=auth,
    verify_ssl_certs=False
)


# response = model.chat(MESSAGE)
# content = response.choices[0].message.content
# # print(content)
# display(Markdown("<blockquote>\n\n"+content))



### Упражнения
1. Предложите промпт, требующий знания информации на текущую дату, на дату несколько лет назад и на достаточно известное историческое событие, сравните и объясните результаты.
2. Проверить качество результата запросов по категориям: математика, естественные науки, гуманитарные науки для разных моделей.
3. Создайте небольшой диалог двух ИИ-персон

In [ ]:
with GigaChat(credentials=auth, model="GigaChat-2", verify_ssl_certs=False) as giga:
  response = giga.chat("Когда случился Коронавирус")
print(response.choices[0].message.content)

Коронавирусная инфекция COVID-19 (вызываемая вирусом SARS-CoV-2) впервые была зафиксирована в конце декабря 2019 года в китайском городе Ухань провинции Хубэй. 

Первые случаи заражения связаны с рынком морепродуктов и живой птицы — предполагается, что источником инфекции могли стать животные. Вскоре после обнаружения заболевания оно быстро распространилось по всему миру, вызвав пандемию, объявленную Всемирной организацией здравоохранения 11 марта 2020 года.

На сегодняшний день пандемия продолжает оставаться актуальной проблемой мировой медицины и общественного здравоохранения.


In [ ]:
from IPython.display import display, Markdown

In [ ]:
messages=[
        Messages(
            role=MessagesRole.USER,
            content="Объясни, что такое AI-agent?"
        ),
]

payload = Chat(
    messages=messages,
    temperature=0.7,
    max_tokens=100,
)

print("Объясни, что такое AI-agent?")
for role in [MessagesRole.ASSISTANT, MessagesRole.USER]:
  with GigaChat(credentials=auth, model="GigaChat-2", verify_ssl_certs=False) as giga:
    response = giga.chat(payload)
    payload.messages.append(Messages(role=role, content=response.choices[0].message.content))
    print("-----------------")
    display(Markdown(response.choices[0].message.content))

Объясни, что такое AI-agent?
-----------------


AI-Agent (искусственный интеллектуальный агент) — это автономная система, способная действовать в окружающей среде самостоятельно, принимая решения и взаимодействуя с внешним миром согласно заданной цели или набору правил. Ключевые характеристики AI-агента включают:

1. **Автономность**: агент действует независимо от человека, выполняя заранее запрограммированные или самообученные алгоритмы действий.
   
2. **Целевая ориентация**: агент имеет цель или набор целей, которые направляют его

-----------------


 поведение. Например, минимизировать затраты, максимизировать прибыль, достичь определённой точки пространства или получить доступ к ресурсам.

3. **Интерактивность**: агент способен воспринимать окружение через сенсорную информацию (визуальные данные, звуковые сигналы, показания датчиков), анализировать её и реагировать соответствующим образом.

4. **Адаптивность**: способность агента корректировать своё поведение на основе опыта взаимодействия с внешней средой, обучаясь новым стратегиям поведения и улучшая эффективность выполнения поставленных задач.

5


# Роли и контекст запроса


Более правиьно формулировать запросы (`payload`) к модели с использованием объекта типа `Chat`.
`Chat` — это объект, описывающий весь чат-запрос к модели. Он содержит:
* `messages` — список сообщений, представляющих историю диалога.
* `temperature` — параметр, управляющий «творчеством» модели:
    * Чем ближе температурак `0`, тем более детерминированный и предсказуемый ответ.
    * Чем ближе температура к `1` (или выше), тем более случайный и разнообразный ответ.
    * Например, значение `0.7` — баланс между креативностью и точностью.
* `max_tokens` — ограничение на длину ответа модели (в токенах). Каждый токен приблизительно одно слово.  


In [ ]:
payload = Chat(
    messages=[
    Messages(role=MessagesRole.USER,
            content=MESSAGE
        )
    ],
    temperature=0.7,
    max_tokens=100,
)

In [ ]:
response = model.chat(MESSAGE)
content = response.choices[0].message.content
# print(content)
display(Markdown("<blockquote>\n\n"+content))

<blockquote>

**Prompt Engineering (инженерия подсказок)** — процесс формирования запросов к искусственному интеллекту таким образом, чтобы получить наиболее точный, полезный и понятный результат от модели.

### Особенности Prompt Engineering:
1. **Четкость формулировки**: запросы должны быть ясными, лаконичными и точно отражающими желаемый результат.
2. **Контекстуальность**: важно учитывать контекст задачи, чтобы подсказка учитывала специфику ситуации и помогала модели лучше понять задание.
3. **Итеративность**: часто требует тестирования разных формулировок запроса и анализа результатов, чтобы найти оптимальную структуру подсказки.

-------------------------------------------------

Сообщения в messages имеют роли (`role`), которые определяют, кто «говорит»:
* `SYSTEM` — задаёт поведение модели. Это инструкция, невидимая пользователю, но влияющая на стиль и содержание ответов.
Пример: «Ты полезный ассистент для тестирования ГигаЧата» — модель будет вести себя как помощник, ориентированный на тестирование.
* `USER` — сообщение от пользователя (реальный вопрос или команда).
* `ASSISTANT` — ответ модели (обычно добавляется автоматически после генерации).

>История диалога формируется последовательным добавлением сообщений с разными ролями.
>
> Модель «помнит» контекст только в рамках одного запроса.

In [ ]:
messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content=(
                "Ты полезный ассистент для тестирования ГигаЧата. Твоя задача отвечать студенту на учебные вопросы\n"
                "## Инструкция\n"
                "Ответ должен подходить студенту начальных курсов бакалавриата, изучающему технологии искусственного интеллекта\n"
                "## Формат ответа\n"
                "Текст и таблицы в формате markdown\n"
            )
        ),
        Messages(
            role=MessagesRole.USER,
            content=MESSAGE
        ),
  ]

In [ ]:
payload = Chat(
    messages=messages,
    temperature=0.7,
    max_tokens=500,
)

response = model.chat(payload)

display(Markdown("<blockquote>\n\n"+response.choices[0].message.content))


<blockquote>

**Определение Prompt Engineering**
Prompt engineering — это процесс создания эффективных и целенаправленных входных данных (prompts) для моделей машинного обучения с целью получения желаемых результатов от нейросетевых систем.

### Особенности Prompt Engineering:

1. **Ясность и Конкретизация**
   - Хорошо сформулированные prompts помогают модели точно понять задачу и избежать неоднозначности или неправильного понимания запроса.
   
2. **Структура и Форматирование**
   - Использование структурированных форматов (например, шаблонов вопросов, инструкций или примеров) позволяет направлять модель к желаемому результату более эффективно.
   
3. **Контекст и Ориентация на Цель**
   - Эффективный prompt учитывает контекст задачи и четко формулирует цель, что помогает модели сосредоточиться именно на необходимых аспектах проблемы.

------------------------------------------------
Для расширения контекста можно дополнять блок `messages` аналогично списку

In [ ]:
payload.messages.append(Messages(role=MessagesRole.USER, content=response.choices[0].message.content))
payload.messages.append(Messages(role=MessagesRole.USER, content="Уточни контент определением ролей"))

response = model.chat(payload)

display(Markdown("<blockquote>\n\n"+response.choices[0].message.content))


<blockquote>

**Prompt Engineering (инженерия подсказок)** — это процесс разработки и оптимизации входных данных (подсказок) для моделей машинного обучения с целью получения наилучших результатов от них.

### Роли участников процесса:
- **Разработчик модели**: создает и обучает саму модель машинного обучения.
- **Инженер по подсказкам (Prompt Engineer)**: разрабатывает и тестирует различные варианты подсказок, подбирая оптимальный запрос для конкретной задачи.
- **Пользователь**: получает итоговый результат

### Упражнения
1. Разработать системный промпт, который на запрос пользователя возвращет суммаризацию запроса и ответ на заопрос.
2. Разработать помпт, который на запрос пользователя всегда будет отчечать в стиле выбранного писателя.
3. Проверить влияние температуры и длины ответа на его качество.
4. Разработать промпт который будет на выходе давать  формат `JSON`, например
```json
{
  "defenition": "Prompt Engineering (инженерия подсказок) — это процесс разработки и оптимизации входных данных (подсказок) для языковых моделей искусственного интеллекта, направленный на получение максимально полезных и точных результатов от модели.",
  "properties": "Четкость формулировки: запрос должен быть четко сформулированным и понятным модели. Контекстуальность: предоставление достаточного контекста помогает модели лучше понять задачу.Гибкость и итерационность: часто требуется несколько попыток",
  "roles": "Пользователь: человек, задающий вопрос или требующий выполнения задачи, получающий результат работы модели. Модель: система искусственного интеллекта, принимающая запросы (подсказки), выполняющая обработку информации."
}```


In [ ]:
messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content=(
                """
                Ты полезный ассистент для тестирования ГигаЧата. Твоя задача отвечать студенту на учебные вопросы
                Отвечай кратко и ёмко!
                ## Шаблон ответа:
                Запрос: {суммаризация запроса пользователя}.
                Ответ: {ответ на запрос со списками, таблицами}.
                Роли: {роли по информации запроса}.
                ## Формат
                Markdown
                """
            )
        ),
        Messages(
            role=MessagesRole.USER,
            content=MESSAGE
        ),
]

In [ ]:
messages=[
        Messages(
            role=MessagesRole.SYSTEM,
            content=(
                """
                Ты полезный ассистент для тестирования ГигаЧата. Твоя задача отвечать студенту на учебные вопросы
                Отвечай кратко и ёмко!
                ## Шаблон ответа:
                {
                  "defenition": "",
                  "properties": "",
                  "roles": ""
                }
                ## Формат
                JSON
                """
            )
        ),
        Messages(
            role=MessagesRole.USER,
            content=MESSAGE
        ),
    ]

In [ ]:
payload = Chat(
    messages=messages,
    temperature=0.7,
    max_tokens=300,
)

response = model.chat(payload)
display(Markdown("<blockquote>\n\n"+response.choices[0].message.content))

<blockquote>

{
  "definition": "Prompt Engineering — это процесс создания эффективных подсказок (prompt) для оптимизации работы моделей машинного обучения, особенно языковых моделей.",
  "properties": [
    "Точность формулировки запроса",
    "Использование специальных маркеров и форматов данных",
    "Тестирование различных вариантов подсказки"
  ],
  "roles": "Помогает достичь более точных и релевантных ответов от модели."
}

# Функции и Актуализация запросов к модели

Языковые модели, включая `GigaChat`, обучены на данных, зафиксированных до определённой даты. Они не знают, что происходит «здесь и сейчас».
Но если дать модели возможность вызвать функцию, которая получит свежие данные (например, через поисковик), — она сможет дать актуальный и точный ответ.

[Механизм вызова функций](https://habr.com/ru/articles/806627/) называется `Function Calling` — механизм, при котором модель:
* Решает, нужно ли вызвать функцию.
* Формирует структурированный запрос к функции (с аргументами).
* Система выполняет функцию.
* Результат возвращается модели, и она формулирует финальный ответ.


__Другими словами__ при использовании `Function Calling` в запрос передаётся не только история сообщений, но и список доступных функций.
Модель анализирует контекст и решает:
Ответить сразу, или вернуть специальное сообщение с `function_call`. Во втором случае будет необходимо вызвать функцию и повторно запросить модель.

> Отметим, что не все модели подддерживают `Function Calling`.

Создадим функцию  `search_ddg`, которая использует библиотеку `ddgs` для получения актуальных результатов поиска.
Где `ddgs` — это библиотека для поиска в `DuckDuckGo`. `DuckDuckGo` выбран, потому что он не требует API-ключа.

In [ ]:
def search_ddg(search_query):
    """Поиск в DuckDuckGo.
        Полезен, когда нужно ответить на вопросы о текущих событиях.
        Входными данными должен быть поисковый запрос."""
    return DDGS().text(search_query, max_results=10)

In [ ]:
results = search_ddg(MESSAGE)
print(results[1])

{'title': 'GISMETEO: Погода в Екатеринбурге сегодня, прогноз погоды ...', 'href': 'https://www.gismeteo.ru/weather-yekaterinburg-4517/', 'body': 'Погода в Екатеринбурге на сегодня , подробный прогноз погоды на сегодня для населенного пункта Екатеринбург , городской округ Екатеринбург , Свердловская область, Россия.'}


-----------------------------------------------------------------------------

Создадим объект `search` типа `Function` - для задания нашей функции `search_ddg` в формате, понятном модели:

* `name` — уникальное имя, по которому модель будет её вызывать.
* `description` — объяснение, когда и зачем использовать эту функцию.
* `parameters` — схема входных данных (в стиле `JSON Schema`):


На входе функции ожидается объект с полем `query` типа `string`. Поле `query` обязательно (`required`).
Подход к описанию аналогичен «документации `API`», которую модель «читает» перед решением вызывать функцию или нет.

In [ ]:
search_func = Function(
    name="duckduckgo_search",
    description="Поиск в DuckDuckGo для получения актуальной информации.",
    parameters=FunctionParameters(
        type="object",
        properties={"query": {"type": "string"}},
        required=["query"],
    ),
)

Для того чтобы модель могла вызывать функцию подадаим `search_func` в качестве значения аргумента `functions` объекта `Chat`.

In [ ]:
messages = [
        Messages(role=MessagesRole.USER, content=MESSAGE)
    ]
chat = Chat(messages=messages, functions=[search_func])
resp = model.chat(chat).choices[0]
message = resp.message
print(message)
resp.finish_reason

role='assistant' content='' function_call=FunctionCall(name='duckduckgo_search', arguments={'query': 'prompt engineering определение и особенности'}) name=None attachments=None data_for_context=None functions_state_id='019ad592-1ac7-7c63-b0a1-f23e9c8cf492' reasoning_content=None id_=None


'function_call'

В следующем коде проверяем, необходим  ли модели вызов функции. Если необходим, то
* формируем запрос `query`
* получаем результат при помощи `search_ddg`
* дополняем сообщение для модели результатом запроса
* формируем окончательный ответ

In [ ]:
# Если модель хочет вызвать функцию
if resp.finish_reason == "function_call":
    func_name = message.function_call.name
    query = message.function_call.arguments["query"]

    # Выполняем функцию
    result = search_ddg(query)

    # Шаг 2: отправляем результат обратно модели
    messages.extend([
        message,  # сообщение с function_call
        Messages(role=MessagesRole.FUNCTION, content=json.dumps({"result": result}, ensure_ascii=False))
    ])
    final_resp = model.chat(Chat(messages=messages)).choices[0]
    response = final_resp.message.content
else:
    # Модель ответила сразу
    print('МОДЕЛЬ ОТВЕТИЛА СРАЗУ')
    response = message.content

display(Markdown("<blockquote>\n\n"+response))

<blockquote>

**Prompt engineering** — это процесс структурирования или составления запроса таким образом, чтобы улучшить качество выходного результата от модели искусственного интеллекта (ИИ). 

### Особенности prompt engineering:

1. **Четкость формулировки:** Важно создавать ясные и конкретные инструкции, чтобы направлять модель к нужному результату.
   
2. **Контекстуальность:** Добавление релевантной информации помогает модели лучше понимать контекст и формировать более точные ответы.

3. **Итеративность:** Процесс часто включает многократное тестирование и уточнение запроса, чтобы достичь наилучшего возможного результата.

### Упражения
1. Измените поле `desription` описания функции, например на
    * "Используй ТОЛЬКО для вопросов о погоде."
    * "Никогда не используй этот поиск."
    * "Это функция для поиска рецептов блюд."
Проверьте как это скажется на результатах.
2. Измените значение `max_results` в диапазоне 1 - 10, провеврьте как это скажется на качестве ответа
3. Добавьте к примеру системный промпт, например "Ты — помощник, который ВСЕГДА ищет информацию в интернете, даже если знаешь ответ."
4. Добавьте функцию текущей даты к списку функций запроса.
```python
   def get_current_date():
        """Возвращает текущую дату в формате ГГГГ-ММ-ДД."""
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d")
```

In [ ]:
def get_current_date():
  from datetime import datetime
  now = datetime.now()
  today = datetime(year=now.year, month=now.month+1, day=5)
  return today.strftime("%Y-%m-%d")

In [ ]:
print(get_current_date())

2025-12-05


In [ ]:
search_func = Function(
    name="get_current_date",
    description="Узнать дату сегодня (сегодняшний день, сегодняшнее число)",
    parameters=FunctionParameters(
        type="object",
        properties={},
        required=[],
    ),
)

In [ ]:
search_func = Function(
    name="duckduckgo_search",
    description="Поиск в DuckDuckGo для получения актуальной информации. Используй ТОЛЬКО для погоды и даты",
    parameters=FunctionParameters(
        type="object",
        properties={"query": {"type": "string"}},
        required=["query"],
    ),
    name="get_current_date",
    description="Узнать дату сегодня (сегодняшний день, сегодняшнее число)",
    parameters=FunctionParameters(
        type="object",
        properties={},
        required=[],
    ),
)

SyntaxError: keyword argument repeated: name (ipython-input-4086365309.py, line 9)

In [ ]:
MESSAGE = "Какое сегодня число?"

messages = [
        Messages(role=MessagesRole.USER, content=MESSAGE)
    ]

chat = Chat(messages=messages, functions=[search_func], max_tokens=100)
resp = model.chat(chat).choices[0]
message = resp.message
print(message)


role='assistant' content='' function_call=FunctionCall(name='get_current_date', arguments={}) name=None attachments=None data_for_context=None functions_state_id='019ad5a6-f225-7be9-b2d1-e7396283682b' reasoning_content=None id_=None


In [ ]:
# Если модель хочет вызвать функцию
if resp.finish_reason == "function_call":
    func_name = message.function_call.name

    # Выполняем функцию
    if func_name == "duckduckgo_search":
      query = message.function_call.arguments["query"]
      result = search_ddg(query)
    elif func_name == "get_current_date":
      result = get_current_date()

    # Шаг 2: отправляем результат обратно модели
    messages.extend([
        message,  # сообщение с function_call
        Messages(role=MessagesRole.FUNCTION, content=json.dumps({"result": result}, ensure_ascii=False))
    ])
    final_resp = model.chat(Chat(messages=messages)).choices[0]
    response = final_resp.message.content
else:
    # Модель ответила сразу
    print('МОДЕЛЬ ОТВЕТИЛА СРАЗУ')
    response = message.content

display(Markdown("<blockquote>\n\n"+response))

<blockquote>

Сегодня 5 декабря 2025 года.

## Упражения 2

Создадим свой калькулятор при помощи функций `GigaChat`

In [ ]:
import re
def safe_calculate(expression: str) -> str:
    """
    Выполняет математическое выражение.
    Поддерживает: +, -, *, /, **, скобки, числа с точкой.
    Безопасен: разрешает ТОЛЬКО математические символы.
    """
    # Разрешённые символы: цифры, операторы, скобки, точка, пробелы
    if not re.fullmatch(r'[\d+\-*/().\s]+', expression):
        return "Ошибка: выражение содержит недопустимые символы."

    try:
        # Ограничиваем сложность (например, не даём выполнить 9**9**9)
        if '^' in expression or len(expression) > 50:
            return "Ошибка: выражение слишком сложное или длинное."

        result = eval(expression, {"__builtins__": {}}, {})
        return str(result)
    except Exception as e:
        return f"Ошибка вычисления: {str(e)}"

In [ ]:
safe_calculate('3*(4+5)**2')

'243'

In [ ]:
calculate_func = Function(
    name="calculate",
    description="Выполняет математические вычисления. Передавай ТОЛЬКО выражение в виде строки, например: '2 + 3 * 4'.",
    parameters=FunctionParameters(
        type="object",
        properties={
            "expression": {
                "type": "string",
                "description": "Математическое выражение (только цифры, +, -, *, /, **, скобки)"
            }
        },
        required=["expression"],
    ),
)

In [ ]:
message = 'Сколько будет 3*(4+5)**2'

messages = [
    Messages(role=MessagesRole.USER, content=message)
]

chat = Chat(messages=messages, functions=[calculate_func])

resp = model.chat(chat).choices[0]
message = resp.message

if resp.finish_reason == "function_call":
    func = message.function_call
    if func.name == "calculate":
        expr = func.arguments.get("expression", "")
        result = safe_calculate(expr)
        # Возвращаем результат модели
        messages.extend([
            message,
            Messages(role=MessagesRole.FUNCTION, content=result)
        ])
        # Получаем финальный ответ
        final = model.chat(Chat(messages=messages)).choices[0]
        response =  final.message.content
else:
    # Модель ответила без вычислений (например, объяснила задачу)
    response = message.content

In [ ]:
display(Markdown("<blockquote>\n\n"+response))

<blockquote>

Решим выражение пошагово:

1. Сначала выполним действие в скобках:  
$4 + 5 = 9$

2. Возведём полученное число в квадрат:  
$9^2 = 81$

3. Теперь умножим результат на 3:  
$3 \cdot 81 = 243$

**Итоговый ответ:** $\mathbf{243}$

__Упражения__
1. Сделайте проверку на sin/cos в функции калькулятора
   
2. Расширьте функционал калькулятора

In [ ]:
calculate_func = Function(
    name="calculate",
    description="Выполняет математические вычисления. Передавай ТОЛЬКО выражение в виде строки, например: '2 + 3 * 4'.",
    parameters=FunctionParameters(
        type="object",
        properties={
            "expression": {
                "type": "string",
                "description": "Математическое выражение (только цифры, +, -, *, /, **, скобки)"
            }
        },
        required=["expression"],
    )
)

sin_func = Function(
    name="sin",
    description="Находит синус выражения. ВАЖНО, сначала высчитай синусы, потом замени синусы на реальные значения и передай аргумент функции калькулятора",
    parameters=FunctionParameters(
        type="object",
        properties={
            "value": {
                "type": "string",
                "description": "Значение угла в радианах"
            }
        },
        required=["value"],
    )
)

In [ ]:
message = 'Сколько будет 55*243?'

messages = [
    Messages(role=MessagesRole.USER, content=message)
]

chat = Chat(messages=messages, functions=[calculate_func, sin_func], max_tokens=100)

resp = model.chat(chat).choices[0]
message = resp.message

if resp.finish_reason == "function_call":
    func = message.function_call
    if func.name == "calculate":
        print("Калькулирую!")
        expr = func.arguments.get("expression", "")
        result = safe_calculate(expr)
    elif func.name == "sin":
        value = func.arguments.get("value", "")
        print("Расчёты!")
        result = eval(f"sin({value})")
    # Возвращаем результат модели
    messages.extend([
        message,
        Messages(role=MessagesRole.FUNCTION, content=result)
    ])
    # Получаем финальный ответ
    final = model.chat(Chat(messages=messages)).choices[0]
    response =  final.message.content
else:
    # Модель ответила без вычислений (например, объяснила задачу)
    response = message.content

Калькулирую!


In [ ]:
print(response)

Произведение чисел 55 и 243 равно $ \fbox{13365} $.


In [ ]:
from math import sin, cos, pi
print(sin(3*pi/2))

-1.0


# <span style="color:red">Опционально.</span> О более продвинутом пути к LLM-приложениям

P.S. По какой-то причине у меня не работает адекватно подгрузка GigaChain

Простой вызов языковой модели — как в примерах выше - подходит для однократных запросов. Но в реальных задачах ИИ-приложения редко ограничиваются одним вопросом. Чаще всего нам нужно:
* вести диалог с памятью,
* вызывать внешние инструменты (`tools`), например поиск, калькулятор, базы данных,
* строить цепочки (`chains`) и комбинировать несколько шагов обработки информации (анализ → поиск → генерация),
* управлять контекстом, форматом вывода и безопасностью.
Для этого созданы спициальные библиотеки для работы с `LLM`, в том числе `LangChain` - фреймворк, который превращает LLM в ИИ-агента.
`LangChain] берёт на себя всю типичную инфраструктуру: управление сообщениями, обработку вызовов функций, повторные запросы, форматирование  и позволяет вам сосредоточиться на логике приложения, а не на ручной сборке JSON-запросов.



[Гигачейн]()(`GigaChain`) - [это переделанная под работу с российскими моделями библиотека `Langchain`](https://github.com/trashchenkov/gigachat_tutorials/blob/main/%D0%B3%D0%B8%D0%B3%D0%B0%D1%87%D0%B5%D0%B9%D0%BD.ipynb). Исходная библиотека `Langchain` позволяет создавать сложные цепочки по обработке данных, поступающих из разных источников, и встраивать в эту обработку большие языковые модели.

Функционал `Langchain` достаточно широкий и постоянно пополняется. Попробуем использовать `langchain_gigachat` (`gigachain`) для того чтобы более компактно работать с моделями

In [1]:
!pip install langchain_gigachat

In [6]:
!pip install langchain

  Using cached langchain_core-1.2.2-py3-none-any.whl.metadata (3.7 kB)
Using cached langchain_core-1.2.2-py3-none-any.whl (476 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-gigachat 0.3.12 requires langchain-core<0.4,>=0.3, but you have langchain-core 1.2.2 which is incompatible.


In [1]:
!pip install langchain

In [5]:
!pip install -U langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain
    Found existing installation: langchain 1.1.3
    Uninstalling langchain-1.1.3:
      Successfully uninstalled langchain-1.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
from langchain_gigachat import GigaChat
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, FunctionMessage

ImportError: cannot import name 'AgentExecutor' from 'langchain.agents' (/usr/local/lib/python3.12/dist-packages/langchain/agents/__init__.py)

In [6]:
import re

In [8]:
# Инициализация модели
llm = GigaChat(
    model="GigaChat-Pro",
    credentials=auth,
    verify_ssl_certs=False,
    streaming=False,
)

NameError: name 'auth' is not defined

В `Langchain` есть классы `HumanMessage`, `SystemMessage` и `AssistantMessage` для удобного представления словарей сообщений.

Например вмето записи сообщания в стиле:
```json
{'role': 'system',
'content': 'Отвечай как бывалый пират. Пусть тебя зовут Генри Морган.'
}
```
теперь можем записать:
```python
SystemMessage(content='Отвечай как бывалый пират. Пусть тебя зовут Генри Морган.')
```

In [9]:
msg = [SystemMessage(content='Отвечай как инженр-датасаинтист с 20 летним опытом. Используй Markdown разметку ответа. Ответ не должен быть длинее 10 строк')]

question = "Какие приемущества может дать langchain в работе с GigaChat"

msg.append(HumanMessage(question))

NameError: name 'SystemMessage' is not defined

In [ ]:
results = llm(msg).content[:600]

display(Markdown("<blockquote>\n\n"+results))

<blockquote>

**Преимущества использования LangChain для работы с GigaChat:**

- **Интеграция и управление знаниями:** позволяет эффективно использовать и обновлять корпоративные данные.
- **Повышение точности ответов:** доступ к актуальным данным улучшает качество генерируемых решений.
- **Автоматизация процессов:** упрощает создание автоматизированных рабочих потоков, снижая трудозатраты.
- **Безопасность данных:** обеспечивает локальное хранение конфиденциальной информации без передачи третьим лицам.
- **Ускорение разработки:** предоставляет готовые компоненты и шаблоны для быстрого прототипирования и вн

При помощи класса `AIMessage` `LangChain` позволяет сохранить ответ на первое сообщение и использовать этот результат при вторичном запросе. В нашем случае попросим уточнить `GigaChat` примеры кода для нашего запроса.

В примере будем использовать метод `invoke` - часть унифицированного `Runnable API (LangChain 0.1.0+)`.
Прямой вызов - устаревший подход, может быть удален в будущем.

In [ ]:
# Получаем ответ
response = llm.invoke(msg)  # Используем invoke вместо прямого вызова
results = response.content[:600]

# Сохраняем ответ в историю
msg.append(AIMessage(content=results))

# Пример продолжения диалога с историей
follow_up_question = "Можешь привести конкретный пример использования цепочки (chain) с GigaChat (langchain_gigachat)?"
msg.append(HumanMessage(content=follow_up_question))

# Получаем ответ с учетом всей истории
follow_up_response = llm.invoke(msg)
msg.append(AIMessage(content=follow_up_response.content))

display(Markdown("<blockquote>\n\n"+follow_up_response.content))

<blockquote>

```python
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Настройка GigaChat через Hugging Face Hub API
llm = HuggingFaceHub(repo_id="gigachat", huggingfacehub_api_token="YOUR_API_TOKEN")

# Создание шаблона запроса
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="{question}"
)

# Создание цепочки (chain)
chain = LLMChain(llm=llm, prompt=prompt_template)

# Запуск цепочки с конкретным вопросом
response = chain.run("Каковы преимущества использования контейнеризации?")
print(response)
```

**Пример демонстрирует:**  
- Интеграцию GigaChat через Hugging Face Hub API;  
- Использование простого цепочного подхода для обработки пользовательских запросов.

Попробуем также в целях демонстрации возможностей `langchain` создать цепочку рассуджений. Для этого воспользуемся специальным классом `PromptTemplate`, который позволяет создавать шаблоны запросов аналогично f-функциям

In [ ]:
from langchain.prompts import PromptTemplate

# Создаем шаблон для Chain of Thought
cot_template = """
Реши задачу шаг за шагом:

Задача: {problem}

Пожалуйста:
1. Сначала пойми, что дано и что нужно найти
2. Разбей решение на логические шаги
3. Выполни вычисления для каждого шага
4. Проверь правильность рассуждений
5. Сформулируй окончательный ответ

"""

cot_prompt = PromptTemplate(
    input_variables=["problem"],
    template=cot_template
)

# Создаем цепочку
cot_chain = LLMChain(llm=llm, prompt=cot_prompt)

# Используем
problems = [
    "В классе 30 учеников. 40% из них - девочки. Сколько мальчиков в классе?",
    "Книга стоит 500 рублей. После скидки цена снизилась на 20%. Сколько стоит книга после скидки?",
]

for problem in problems:
    result = cot_chain.run(problem=problem)
    display(Markdown(f"**Задача:\n** {problem}\n----------------------------"))
    display(Markdown(f"**Решение:\n\n** {result}"))
    print("---")

**Задача:
** В классе 30 учеников. 40% из них - девочки. Сколько мальчиков в классе?
----------------------------

**Решение:

** ### Шаг 1. Понимание условия задачи

Дано:
- Всего учеников в классе — 30 человек.
- Девочки составляют $40\%$ от общего числа учеников.

Нужно найти:
- Количество мальчиков в классе.

---

### Шаг 2. Логическая последовательность решения

1. Найдём количество девочек (это будет $40\%$ от всех учеников).
2. Отнимем полученное число девочек от общего количества учеников, чтобы узнать количество мальчиков.

---

### Шаг 3. Вычисления по шагам

#### Шаг 3.1. Нахождение количества девочек

Общее количество учеников составляет $30$. Человек.

Найдем $40\%$ от $30$:
\[
\frac{40}{100} \times 30 = 0.4 \times 30 = 12\ (\text{девочек})
\]

Таким образом, в классе $12$ девочек.

#### Шаг 3.2. Нахождение количества мальчиков

Из общего числа учеников вычитаем количество девочек:
\[
30 - 12 = 18\ (\text{мальчиков})
\]

Получили, что в классе $18$ мальчиков.

---

### Шаг 4. Проверка правильности

Проверим корректность наших вычислений:

- $40\%$ от $30$ действительно равно $12$.
- Если отнять $12$ девочек от $30$, останется ровно $18$ мальчиков.

Вычисления верны.

---

### Ответ:

В классе $18$ мальчиков.

---


**Задача:
** Книга стоит 500 рублей. После скидки цена снизилась на 20%. Сколько стоит книга после скидки?
----------------------------

**Решение:

** ### Шаг 1: Понимание условия задачи

Дано:
- Исходная стоимость книги — 500 рублей.
- Размер скидки — 20%.

Нужно найти:
- Стоимость книги после применения скидки.

---

### Шаг 2: Логическая разбивка решения

Для нахождения стоимости товара после скидки выполним следующие шаги:

1. Найдём размер скидки (в рублях).
2. Вычтем найденную скидку из исходной цены.

---

### Шаг 3: Вычисления

#### 1. Нахождение размера скидки:
Скидка составляет 20% от 500 рублей.

$500 \times \frac{20}{100} = 500 \times 0.2 = 100$ рублей.

#### 2. Вычитание скидки из первоначальной цены:
Новая цена будет равна исходной цене минус скидка:

$500 - 100 = 400$ рублей.

---

### Шаг 4: Проверка правильности рассуждений

Проверим, действительно ли получилась правильная скидка:
- Если отнять 100 рублей от 500, получаем 400.
- Это соответствует уменьшению цены на 20%, так как $\frac{100}{500} \times 100\% = 20\%$.

Таким образом, рассуждения верны.

---

### Ответ:

После скидки книга стоит **400 рублей**.

---


### Упражнение:

Проверьте качество работы цепочки рассуждений для разных категорий вопросов

In [ ]:
problems = [
    "В классе 30 учеников. 40% из них - девочки. Сколько мальчиков в классе?",
    "Книга стоит 500 рублей. После скидки цена снизилась на 20%. Сколько стоит книга после скидки?",
]

for problem in problems:
    result = cot_chain.run(problem=problem)
    display(Markdown(f"**Задача:\n** {problem}\n----------------------------"))
    display(Markdown(f"**Решение:\n\n** {result}"))
    print("---")